Description: in this notebook we show how to manually inspect the warnings of your test run.

# Import Libraries

In [1]:
import project_path

import matplotlib.pyplot as plt
import os
import pandas as pd
import pprint
import seaborn as sns
import sqlite3 as sl
from tqdm import tqdm
from typing import List, Any, Tuple, Dict
pp = pprint.PrettyPrinter(indent=4)

from lib.detectors import KS_Detector
from lib.inspector import Inspector
from lib.inspector import OneNightStandInspector
from lib.inspector import retrieve_relevant_file_paths
from lib.inspector import read_program
from lib.inspector import inspec_column_of
from lib.inspector import copy_bug
from lib.inspector import get_alarms_with_method
from lib.qmt import execute_programs
from lib.utils import iterate_over
from lib.utils import load_config_and_check

%load_ext autoreload
%autoreload 2

# Parameters
Insert here the path to the relevant folders related to your test run.

In [2]:
EXP_FOLDER = "qmt_v53"
PATH_DATASET = "../miscellanea/table_artifacts/Qiskit_Bug_Dataset - main_dataset.csv"

# READ THE SQLITE DATABASE WITH CRASHES
SQL_PATH = os.path.join("..", "data", EXP_FOLDER, "qfl.db")
SQL_CONNECTION = sl.connect(SQL_PATH)

# SETTING FOR DIVERGENCE DETECTION
PVAL_COL = "divergence.ks.p-value" 
ALPHA_LEVEL = 0.05

# Read Program Pairs

In [3]:
# READ CRASHES
df_crash = pd.read_sql("""SELECT * from CRASHDATA""", SQL_CONNECTION)
# READ COMPLETE EXECUTIONS
df_data = pd.read_sql("""SELECT * from QFLDATA""", SQL_CONNECTION)
# CONSIDER BOTH
df_generated_programs = pd.concat(
    [df_data.assign(generated_a_crash=False),
     df_crash.assign(generated_a_crash=True)], axis=0)
# READ ANNOTATED BUGS
df_bugs = pd.read_csv(PATH_DATASET)
df_bugs = df_bugs.fillna("-")

print(f"Total (potentially divergent) comparisons: {len(df_data)}")
print(f"Total crash in programs: {len(df_crash)}")
print(f"\tcrashing source: {len(df_crash[~df_crash['exceptions.source'].isna()])}")
print(f"\tcrashing follow-up: {len(df_crash[~df_crash['exceptions.followup'].isna()])}")

Total (potentially divergent) comparisons: 6417
Total crash in programs: 1943
	crashing source: 0
	crashing follow-up: 1943


In [10]:
print("Some example crash messages:")
list(df_crash["exceptions.followup"].head(5))

Some example crash messages:


["'Mismatch between run_config.parameter_binds and all circuit parameters. Parameter binds: [] Circuit parameters: [ParameterView([Parameter(p_0c596f), Parameter(p_597708), Parameter(p_7ef6c2), Parameter(p_bb82fc)])]'",
 "'Cannot represent circuits with unbound parameters in OpenQASM 2.'",
 "'Mismatch between run_config.parameter_binds and all circuit parameters. Parameter binds: [] Circuit parameters: [ParameterView([Parameter(p_2f6fa1), Parameter(p_31d9e7), Parameter(p_deb987), Parameter(p_f98523)])]'",
 "'Mismatch between run_config.parameter_binds and all circuit parameters. Parameter binds: [] Circuit parameters: [ParameterView([Parameter(p_223c67), Parameter(p_2cb1e6), Parameter(p_5a5ff6), Parameter(p_8e5709)])]'",
 '"Cannot find gate definition for \'c3sx\', line 10 file "']

# Query programs [specific crash message]

In [11]:
QUERY_MESSAGE_CONTENT = "'Cannot represent circuits with unbound parameters in OpenQASM 2.'"  # change here

df_specific_class = df_crash[df_crash["exceptions.followup"].str.contains(QUERY_MESSAGE_CONTENT)]

print(f"We have {len(df_specific_class)} program pairs satisfying the query.")
PROGRAM_ID = df_specific_class.sort_values(by=[
    "source.n_qubits", "source.n_ops"
]).iloc[0]["program_id"]
print(f"The PROGRAM_ID of the first program pair is this: {PROGRAM_ID} ")

We have 58 program pairs satisfying the query.
The PROGRAM_ID of the first program pair is this: fc379c309d064bf5a78029c31d25b873 


# Inspect programs

In [12]:
PROGRAM_ID = PROGRAM_ID # CHANGE THIS MANUALLY 
# OR LEAVE IT THE SAME IF YOU WANT THE ONE FROM THE PREVIOUS STEP

In [13]:
BUG_PATHS = retrieve_relevant_file_paths(
    experiment_folder=EXP_FOLDER,
    program_id=PROGRAM_ID
)
print("PATH of {PROGRAM_ID} pair:")
pp.pprint(BUG_PATHS)
print("\nMetamorphic transformations used:")
pp.pprint(inspec_column_of(
    df=df_generated_programs, 
    program_id=PROGRAM_ID, 
    target_col="followup.metamorphic_transformations"
))

from termcolor import colored
print("READING SOURCE AND FOLLOWUP PROGRAMS")
print('# SOURCE')
read_program(path=BUG_PATHS['source'], color='red')
print("#" + "-" * 80)
print('# FOLLOWUP')
read_program(path=BUG_PATHS['followup'], color='blue')

PATH of {PROGRAM_ID} pair:
{   'followup': '../data/qmt_v53/programs/followup/fc379c309d064bf5a78029c31d25b873.py',
    'metadata': '../data/qmt_v53/programs/metadata/fc379c309d064bf5a78029c31d25b873.json',
    'metadata_exec': '../data/qmt_v53/programs/metadata_exec/fc379c309d064bf5a78029c31d25b873.json',
    'source': '../data/qmt_v53/programs/source/fc379c309d064bf5a78029c31d25b873.py'}

Metamorphic transformations used:
n hits: 1
['ChangeBackend', 'ToQasmAndBack', 'InjectParameters']
None
READING SOURCE AND FOLLOWUP PROGRAMS
# SOURCE

# SECTION
# NAME: PROLOGUE

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library.standard_gates import *
from qiskit.circuit import Parameter

# SECTION
# NAME: CIRCUIT

qr = QuantumRegister(4, name='qr')
cr = ClassicalRegister(4, name='cr')
qc = QuantumCircuit(qr, cr, name='qc')
qc.append(RZGate(6.163759533339787), qargs=[qr[1]], cargs=[])
qc.append(RZZGate(4.066449154047175), qargs=[qr[2], q

"# SECTION\n# NAME: PROLOGUE\n\nimport qiskit\nfrom qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister\nfrom qiskit.circuit.library.standard_gates import *\nfrom qiskit.circuit import Parameter\n# SECTION\n# NAME: PARAMETERS\n# SECTION\n# NAME: PARAMETERS\np_cc364f = Parameter('p_cc364f')\np_8e78eb = Parameter('p_8e78eb')\np_785a6a = Parameter('p_785a6a')\np_885be7 = Parameter('p_885be7')\n\n# SECTION\n# NAME: CIRCUIT\nqr = QuantumRegister(4, name='qr')\ncr = ClassicalRegister(4, name='cr')\nqc = QuantumCircuit(qr, cr, name='qc')\nqc.append(RZGate(p_785a6a), qargs=[qr[1]], cargs=[])\nqc.append(RZZGate(p_cc364f), qargs=[qr[2], qr[3]], cargs=[])\nqc.append(iSwapGate(), qargs=[qr[2], qr[3]], cargs=[])\nqc.append(CSXGate(), qargs=[qr[1], qr[0]], cargs=[])\nqc.append(XGate(), qargs=[qr[2]], cargs=[])\nqc.append(CUGate(0.5112149185250571, p_885be7, 2.3864521352475245, \n    5.987304452123941), qargs=[qr[0], qr[2]], cargs=[])\nqc.append(CU1Gate(p_8e78eb), qargs=[qr[3], qr[0]], c